In [1]:
import ee, datetime
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
geometry = ee.Geometry.Polygon([[[77.42117255516425,23.20909792227606],[77.42057174034491,23.20799353262676],[77.42040007896796,23.207520219982936],[77.4200996715583,23.2069285768207],[77.41949885673895,23.207125791499145],[77.41902678795233,23.207007462726995],[77.41889804191962,23.206258044736966],[77.41868346519843,23.20550862254438],[77.41868346519843,23.204995857570196],[77.41851180382147,23.204680308915954],[77.41816848106757,23.203970321719954],[77.41885512657538,23.20314199855786],[77.41928428001776,23.202787001345527],[77.4204429943122,23.201840337503217],[77.42082923241034,23.201406447668166],[77.42125838585272,23.20089366695645],[77.42173045463934,23.20101200114144],[77.4224600154914,23.20140644766817],[77.42327540703192,23.20152478139923],[77.42361872978583,23.20187978196384],[77.42366164513007,23.202313670262438],[77.42413371391669,23.202589780264603],[77.42464869804755,23.203209392314182],[77.42507785148993,23.203761608047508],[77.42524951286688,23.20451104003633],[77.42451995201483,23.20569434514694],[77.42387622185126,23.206207107441198],[77.42348998375311,23.20707485453413],[77.42306083031075,23.208179251775938],[77.42284625358954,23.208692004535596],[77.42207377739327,23.20853423466525],[77.42143004722968,23.20837646460864],[77.42117255516425,23.20909792227606]]])

In [4]:
def get_data(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)
# NDCI calculation
    def calculate_NDCI(image):
        ndci = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
        return image.addBands(ndci)
    ndci = sentinel_AOI.map(calculate_NDCI)  

# NDCI mean
    def calculate_mean_NDCI(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDCI']),
                           geometry = AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
        return mean.get('NDCI').getInfo()    
    
# NDCI mean collection    
    Images_ndci = ndci.select('NDCI').toList(ndci.size())
    ndci_coll = []
    for i in range(Images_ndci.length().getInfo()):
        image = ee.Image(Images_ndci.get(i-1))
        temp_ndci = calculate_mean_NDCI(image)
        ndci_coll.append(temp_ndci)
# Dates collection
    dates = np.array(ndci.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for chlorophyll
    df = pd.DataFrame(ndci_coll, index = day, columns = ['Chlorophyll'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df


In [5]:
# Calling the function
df1 = get_data('2019-01-01', '2019-02-10')
df1.to_csv('ndci.csv')